## Python Socket 双向通讯

In [1]:
import socket
import threading
import time

# 服务器逻辑
def server_thread(stop_event):
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(('0.0.0.0', 8000))  # 绑定到0.0.0.0和端口8000
    server.listen(1)  # 监听最大一个连接，避免资源浪费。
    print("服务器已启动，等待连接...")

    def receive_data(soc):
        while not stop_event.is_set():
            try:
                data = soc.recv(1024)
                if not data:
                    break
                print(f"接收到客户端数据: {data.decode('utf8')}")
                if data.decode('utf8') == 'exit':
                    print("客户端要求断开连接")
                    break
            except Exception as e:
                print(f"接收数据出错: {e}")
                break

    def send_data(soc):
        while not stop_event.is_set():
            re_data = input("服务器回复: ")
            if re_data == 'exit':
                soc.send('exit'.encode('utf-8'))
                print("服务器断开连接")
                break
            try:
                soc.send(re_data.encode("utf-8"))
            except Exception as e:
                print(f"发送数据出错: {e}")
                break

    while True:
        # 接受客户端连接
        soc, addr = server.accept()
        print(f"客户端连接: {addr}")

        # 启动接收数据和发送数据的线程
        receive_thread = threading.Thread(target=receive_data, args=(soc,))
        send_thread = threading.Thread(target=send_data, args=(soc,))
        
        receive_thread.start()
        send_thread.start()

        # 等待线程完成
        receive_thread.join()
        send_thread.join()

        # 关闭连接
        soc.close()
        if stop_event.is_set():
            break

    server.close()

# 客户端逻辑
def client_thread(stop_event):
    # 客户端连接到服务器
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(('127.0.0.1', 8000))

    def receive_data():
        while not stop_event.is_set():
            try:
                data = client.recv(1024)
                if not data:
                    break
                print(f"接收到服务器响应: {data.decode('utf8')}")
                if data.decode('utf8') == 'exit':
                    print("服务器要求断开连接")
                    break
            except Exception as e:
                print(f"接收数据出错: {e}")
                break

    def send_data():
        while not stop_event.is_set():
            re_data = input("客户端发送: ")
            if re_data == 'exit':
                client.send('exit'.encode('utf-8'))
                print("客户端断开连接")
                break
            try:
                client.send(re_data.encode("utf8"))
            except Exception as e:
                print(f"发送数据出错: {e}")
                break

    # 启动接收数据和发送数据的线程
    receive_thread = threading.Thread(target=receive_data)
    send_thread = threading.Thread(target=send_data)

    receive_thread.start()
    send_thread.start()

    # 等待线程完成
    receive_thread.join()
    send_thread.join()

    # 关闭连接
    client.close()

# 创建一个事件用于停止线程
stop_event = threading.Event()

# 创建并启动线程
server_t = threading.Thread(target=server_thread, args=(stop_event,))
client_t = threading.Thread(target=client_thread, args=(stop_event,))

# 启动线程
server_t.start()
time.sleep(1)  # 等待服务器启动
client_t.start()

# 等待线程完成
client_t.join()
server_t.join()

print("客户端和服务器通信完成")

服务器已启动，等待连接...
客户端连接: ('127.0.0.1', 48226)
接收到客户端数据: 1
接收到服务器响应: 2
接收到服务器响应: 1
接收到服务器响应: 3


: 